In [4]:
import requests

def check_wikidata_response(qid):
    # Wikidata API 요청 URL
    entity_url = f"https://www.wikidata.org/wiki/Special:EntityData/{qid}.json"
    
    # Wikidata API 요청
    response = requests.get(entity_url)
    
    # 응답 상태 코드 확인
    if response.status_code == 200:
        try:
            entity_data = response.json()  # JSON으로 변환
            # QID가 유효하다면 sitelinks에서 enwiki 문서 제목 확인
            if 'entities' in entity_data and qid in entity_data['entities']:
                title = entity_data['entities'][qid].get('sitelinks', {}).get('enwiki', {}).get('title')
                if title:
                    print(f"문서 제목: {title}")
                    return title
                else:
                    print("영어 위키피디아 문서가 존재하지 않습니다.")
            else:
                print("유효하지 않은 QID입니다.")
        except ValueError:
            print("JSON 형식으로 변환할 수 없습니다.")
    else:
        print(f"Wikidata API 응답 실패: {response.status_code}")

# QID 예시
check_wikidata_response("Q42")  # Q42는 Douglas Adams를 나타내는 QID입니다.


문서 제목: Douglas Adams


'Douglas Adams'

In [7]:
import requests
import datetime

def get_wikipedia_views(qid, years=3):
    # Wikidata API 요청 URL 및 제목 확인
    entity_url = f"https://www.wikidata.org/wiki/Special:EntityData/{qid}.json"
    response = requests.get(entity_url)
    
    # Wikidata API 응답 확인
    if response.status_code == 200:
        try:
            entity_data = response.json()  # JSON으로 변환
            if 'entities' in entity_data and qid in entity_data['entities']:
                title = entity_data['entities'][qid].get('sitelinks', {}).get('enwiki', {}).get('title')
                if not title:
                    print("영어 위키피디아 문서가 존재하지 않습니다.")
                    return
            else:
                print("유효하지 않은 QID입니다.")
                return
        except ValueError:
            print("JSON 형식으로 변환할 수 없습니다.")
            return
    else:
        print(f"Wikidata API 응답 실패: {response.status_code}")
        return
    
    # 조회수를 가져오기 위한 Wikimedia API 호출 설정
    end_date = datetime.datetime.now()
    start_date = end_date - datetime.timedelta(days=years*365)  # 3년 전 시작 날짜
    start_str = start_date.strftime('%Y%m%d')
    end_str = end_date.strftime('%Y%m%d')

    # Wikimedia API 요청 URL 설정
    url = f"https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/{title}/monthly/{start_str}/{end_str}"
    views_response = requests.get(url)
    
    # Wikimedia API 응답 확인 및 조회수 출력
    if views_response.status_code == 200:
        views_data = views_response.json().get('items', [])
        if views_data:
            print(f"최근 {years}년간 월별 조회수 ({title}):")
            for item in views_data:
                print(f"{item['timestamp'][:6]}: {item['views']} views")
        else:
            print("조회수 데이터가 존재하지 않습니다.")
    else:
        print("조회수 데이터를 가져오는 데 실패했습니다.", views_response.status_code)

# 사용 예시
get_wikipedia_views("Q42")  # 예시로 Q42(QID) 사용


조회수 데이터를 가져오는 데 실패했습니다. 403


In [9]:
import requests
import datetime

def get_wikipedia_views(qid, years=3):
    headers = {
        "User-Agent": "MyApp/1.0 (https://mywebsite.com; myemail@example.com)"
    }
    
    try:
        # Wikidata API 요청 URL 및 제목 확인
        entity_url = f"https://www.wikidata.org/wiki/Special:EntityData/{qid}.json"
        response = requests.get(entity_url, headers=headers)
        response.raise_for_status()  # HTTP 오류 발생 시 예외 처리
        
        # JSON 응답 파싱
        entity_data = response.json()
        
        # 영어 위키피디아 문서 제목 확인
        if 'entities' in entity_data and qid in entity_data['entities']:
            title = entity_data['entities'][qid].get('sitelinks', {}).get('enwiki', {}).get('title')
            if not title:
                print("영어 위키피디아 문서가 존재하지 않습니다.")
                return
        else:
            print("유효하지 않은 QID입니다.")
            return
        
    except requests.exceptions.RequestException as e:
        print(f"Wikidata API 요청 오류: {e}")
        return
    except ValueError:
        print("JSON 형식으로 변환할 수 없습니다.")
        return
    
    # 조회수를 가져오기 위한 Wikimedia API 호출 설정
    end_date = datetime.datetime.now()
    start_date = end_date - datetime.timedelta(days=years*365)
    start_str = start_date.strftime('%Y%m%d')
    end_str = end_date.strftime('%Y%m%d')

    # Wikimedia API 요청 URL 설정
    url = f"https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/{title}/monthly/{start_str}/{end_str}"
    try:
        views_response = requests.get(url, headers=headers)
        views_response.raise_for_status()  # HTTP 오류 발생 시 예외 처리
        
        # JSON 응답 파싱
        views_data = views_response.json().get('items', [])
        if views_data:
            print(f"최근 {years}년간 월별 조회수 ({title}):")
            for item in views_data:
                print(f"{item['timestamp'][:6]}: {item['views']} views")
        else:
            print("조회수 데이터가 존재하지 않습니다.")
    
    except requests.exceptions.RequestException as e:
        print(f"Wikimedia API 요청 오류: {e}")

# 사용 예시
get_wikipedia_views("Q42")  # 예시로 Q42(QID) 사용


최근 3년간 월별 조회수 (Douglas Adams):
202111: 35481 views
202112: 38988 views
202201: 44538 views
202202: 41714 views
202203: 62732 views
202204: 49400 views
202205: 54278 views
202206: 40787 views
202207: 44467 views
202208: 47280 views
202209: 43272 views
202210: 43229 views
202211: 42249 views
202212: 41597 views
202301: 44801 views
202302: 41504 views
202303: 53895 views
202304: 43802 views
202305: 54156 views
202306: 45858 views
202307: 49070 views
202308: 45749 views
202309: 54664 views
202310: 48127 views
202311: 55814 views
202312: 55058 views
202401: 56329 views
202402: 45579 views
202403: 57121 views
202404: 50782 views
202405: 55162 views
202406: 46264 views
202407: 47216 views
202408: 48043 views
202409: 60961 views
202410: 52929 views
202411: 5467 views


In [10]:
import requests
import datetime

def get_wikipedia_views(qid, years=3):
    headers = {
        "User-Agent": "MyApp/1.0 (https://mywebsite.com; myemail@example.com)"
    }
    
    try:
        # Wikidata API 요청 URL 및 제목 확인
        entity_url = f"https://www.wikidata.org/wiki/Special:EntityData/{qid}.json"
        response = requests.get(entity_url, headers=headers)
        response.raise_for_status()  # HTTP 오류 발생 시 예외 처리
        
        # JSON 응답 파싱
        entity_data = response.json()
        
        # 영어 위키피디아 문서 제목 확인
        if 'entities' in entity_data and qid in entity_data['entities']:
            title = entity_data['entities'][qid].get('sitelinks', {}).get('enwiki', {}).get('title')
            if not title:
                print("영어 위키피디아 문서가 존재하지 않습니다.")
                return None
        else:
            print("유효하지 않은 QID입니다.")
            return None
        
    except requests.exceptions.RequestException as e:
        print(f"Wikidata API 요청 오류: {e}")
        return None
    except ValueError:
        print("JSON 형식으로 변환할 수 없습니다.")
        return None
    
    # 조회수를 가져오기 위한 Wikimedia API 호출 설정
    end_date = datetime.datetime.now()
    start_date = end_date - datetime.timedelta(days=years*365)
    start_str = start_date.strftime('%Y%m%d')
    end_str = end_date.strftime('%Y%m%d')

    # Wikimedia API 요청 URL 설정
    url = f"https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/{title}/monthly/{start_str}/{end_str}"
    try:
        views_response = requests.get(url, headers=headers)
        views_response.raise_for_status()  # HTTP 오류 발생 시 예외 처리
        
        # JSON 응답 파싱 및 전체 조회수 합산
        views_data = views_response.json().get('items', [])
        total_views = sum(item['views'] for item in views_data) if views_data else 0
        
        return total_views  # 전체 조회수 반환
    
    except requests.exceptions.RequestException as e:
        print(f"Wikimedia API 요청 오류: {e}")
        return None

# 사용 예시
total_views = get_wikipedia_views("Q42")  # 예시로 Q42(QID) 사용
if total_views is not None:
    print(f"최근 3년간 전체 조회수: {total_views} views")
else:
    print("조회수 데이터를 가져오지 못했습니다.")


최근 3년간 전체 조회수: 1748363 views


In [ ]:
import requests
import datetime

def get_wikipedia_views(qid, years=3):
    headers = {
        "User-Agent": "MyApp/1.0 (https://mywebsite.com; myemail@example.com)"
    }

    # Wikidata API 요청 URL 및 제목 확인
    entity_url = f"https://www.wikidata.org/wiki/Special:EntityData/{qid}.json"
    response = requests.get(entity_url, headers=headers)
    
    # 문서 제목 추출
    entity_data = response.json()
    title = entity_data['entities'][qid].get('sitelinks', {}).get('enwiki', {}).get('title', None)
    if not title:
        return -1

    # 조회수를 가져오기 위한 Wikimedia API 호출 설정
    end_date = datetime.datetime.now()
    start_date = end_date - datetime.timedelta(days=years*365)
    start_str = start_date.strftime('%Y%m%d')
    end_str = end_date.strftime('%Y%m%d')

    # Wikimedia API 요청 URL 설정
    url = f"https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/{title}/monthly/{start_str}/{end_str}"
    views_response = requests.get(url, headers=headers)

    # 전체 조회수 합산 후 반환
    views_data = views_response.json().get('items', [])
    total_views = sum(item['views'] for item in views_data) if views_data else -1
    
    return total_views

# 사용 예시
total_views = get_wikipedia_views("Q42")  # 예시로 Q42(QID) 사용

최근 3년간 전체 조회수: 1748363 views


In [31]:
import pandas as pd
from tqdm import tqdm
import numpy as np

df = pd.read_excel("one_hop/one_hop_sbj.xlsx")

sbj_list = df[['index', 'subject_id']]
sbj_list['view'] = np.zeros(len(df))
sbj_list.head()
sbj_list.loc[0, 'view']

/tmp/ipykernel_433205/3413113315.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sbj_list['view'] = np.zeros(len(df))


0.0

In [33]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm  # 변경된 부분

df = pd.read_excel("one_hop/one_hop_sbj.xlsx")
df.head()

a = 0
sbj_list = df[['index', 'subject_id']].copy()  # 원본 DataFrame 보호
sbj_list['view'] = np.zeros(len(df))

for idx, qid in tqdm(enumerate(sbj_list['subject_id'])):
    total_views = get_wikipedia_views(qid)
    sbj_list.iloc[idx, sbj_list.columns.get_loc('view')] = total_views  # 변경된 부분
    if total_views == -1:
        a += 1

print(a)


0it [00:00, ?it/s]

KeyboardInterrupt: 